# Q0 — Data Preprocessing (主线前置)

目的：把官方提供的 wide 表（每个选手一行、多周多列）转换为后续建模统一使用的两张 canonical 数据表：
- `data/processed/dwts_weekly_panel.csv`（周×选手面板：评委分/百分位/排名、是否活跃、是否淘汰等）
- `data/processed/dwts_season_features.csv`（季×选手静态特征）

对应主线代码：
- `src/mcm2026/pipelines/mcm2026c_q0_build_weekly_panel.py`

后续依赖：Q1/Q2/Q3/Q4 都依赖 `dwts_weekly_panel.csv`。

运行建议（仓库根目录）：
```bash
uv run python run_all.py
```


## 这一步我们到底在做什么？（口语版）

你可以把官方数据想成一张“很宽的成绩单”：
- **每一行** = 一个季度里的一个选手
- **每一列** = 某一周某一个评委的分数（以及一些静态信息）

这种 wide 表对人来说好看，但对建模/画图很难用。

所以 Q0 的目标很简单：把它改造成后续所有题都能直接用的两张“标准表”。

### A) `dwts_weekly_panel.csv`（周×选手面板）
后面所有分析基本都围绕它。

你可以把它理解成“每周每个还在场上的选手，都有一行记录”：
- **技术线（评委）**：`judge_score_total`、`judge_score_pct`、`judge_rank`
- **状态**：`active_flag`（本周是否还在比赛）、`exit_type`（离开类型）

### B) `dwts_season_features.csv`（季×选手静态特征）
你可以把它理解成“选手的个人简历/背景信息”，比如：年龄、行业、地区等。

### 为什么后续必须依赖这一步？
因为 Q1/Q2/Q3/Q4 都需要一个统一口径的：
- 每周谁在场
- 每周评委如何给分
- 每周谁被淘汰/离开

如果这一步口径不统一，后面所有题都会出现“同一件事在不同脚本里算出来不一样”的灾难。

In [ ]:
from __future__ import annotations

import sys
from pathlib import Path

import pandas as pd

def _find_repo_root(start: Path | None = None) -> Path:
    here = Path.cwd() if start is None else Path(start)
    for p in [here, *here.parents]:
        if (p / 'pyproject.toml').exists():
            return p
    return here

REPO_ROOT = _find_repo_root()
SRC_DIR = REPO_ROOT / 'src'
if str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))

REPO_ROOT


In [ ]:
# 工具函数：在 notebook 里自动展示 outputs/figures 下的图片（优先 TIFF，其次提示 EPS 路径）
from pathlib import Path


def show_saved_figures(question: str, stems: list[str]) -> None:
    """Display figures saved by visualization scripts.

    - Prefer TIFF (good for notebook preview)
    - EPS usually cannot be previewed in Jupyter without额外系统依赖，所以这里只打印路径
    """

    try:
        from IPython.display import Image, display
    except Exception:
        print('⚠️ IPython not available; cannot display images inline.')
        return

    q = question.strip().lower()
    tiff_dir = Path(REPO_ROOT) / 'outputs' / 'figures' / q / 'tiff'
    eps_dir = Path(REPO_ROOT) / 'outputs' / 'figures' / q / 'eps'

    shown = 0
    for stem in stems:
        tiff_fp = tiff_dir / f'{stem}.tiff'
        eps_fp = eps_dir / f'{stem}.eps'

        if tiff_fp.exists():
            display(Image(filename=str(tiff_fp)))
            shown += 1
        elif eps_fp.exists():
            print('EPS figure exists (not previewed inline):', eps_fp)

    if shown == 0:
        print('⚠️ No TIFF figures found under:', tiff_dir)
        print('   If you already generated EPS/TIFF, check filenames and rerun visualization.')


# 示例：show_saved_figures('q0', ['some_figure_stem'])

## 一键运行（推荐给新队友/论文手）

如果你不想按 cell 一步步跑，只想“点一下就把 Q0 跑完并看到关键检查结果”，运行下面这一格即可。

它会做：
- 运行 Q0 生成两张标准表
- 打印 shape / 范围
- 画一个很直观的 sanity plot：每周 active 选手数应该整体下降

如果这一格能顺利跑完且曲线看起来正常，说明 **Q0 的数据口径基本没问题**。

In [ ]:
import traceback

try:
    import sys
    from pathlib import Path

    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt

    # ---- Make this cell self-contained (no dependency on earlier cells) ----
    def _find_repo_root(start: Path | None = None) -> Path:
        here = Path.cwd() if start is None else Path(start)
        for p in [here, *here.parents]:
            if (p / 'pyproject.toml').exists():
                return p
        return here

    if 'REPO_ROOT' not in globals() or not isinstance(globals().get('REPO_ROOT'), Path):
        REPO_ROOT = _find_repo_root()

    SRC_DIR = REPO_ROOT / 'src'
    if str(SRC_DIR) not in sys.path:
        sys.path.insert(0, str(SRC_DIR))

    from mcm2026.pipelines import mcm2026c_q0_build_weekly_panel

    q0_out = mcm2026c_q0_build_weekly_panel.run()
    print('✅ Q0 done:', q0_out)

    weekly = pd.read_csv(q0_out.weekly_panel_csv)
    features = pd.read_csv(q0_out.season_features_csv)

    print('weekly_panel shape:', weekly.shape)
    print('season_features shape:', features.shape)

    print('season range:', int(pd.to_numeric(weekly['season'], errors='coerce').min()), '→', int(pd.to_numeric(weekly['season'], errors='coerce').max()))
    print('week range:', int(pd.to_numeric(weekly['week'], errors='coerce').min()), '→', int(pd.to_numeric(weekly['week'], errors='coerce').max()))

    w = weekly.copy()
    if 'active_flag' in w.columns:
        w['active_flag'] = w['active_flag'].astype(bool)

    n_active = (
        w[w['active_flag']]
        .groupby(['season', 'week'], sort=True)['celebrity_name']
        .nunique()
        .reset_index(name='n_active')
    )

    seasons = sorted(pd.to_numeric(n_active['season'], errors='coerce').dropna().astype(int).unique())
    if seasons:
        sample = [seasons[0], seasons[len(seasons)//2], seasons[-1]]
    else:
        sample = []

    fig, ax = plt.subplots(figsize=(10, 4))
    for s in sample:
        g = n_active[n_active['season'].astype(int) == int(s)].sort_values('week')
        ax.plot(g['week'], g['n_active'], marker='o', linewidth=2, label=f'Season {s}')

    ax.set_title('Quick sanity: active contestants should decrease over weeks')
    ax.set_xlabel('Week')
    ax.set_ylabel('# Active contestants')
    ax.grid(True, alpha=0.3)
    ax.legend()
    plt.tight_layout()
    plt.show()

except Exception as e:
    print('❌ One-click Q0 failed:', e)
    traceback.print_exc()

✅ Q0 done: Q0Outputs(weekly_panel_csv=WindowsPath('D:/MathematicalModeling/2026mcm/data/processed/dwts_weekly_panel.csv'), season_features_csv=WindowsPath('D:/MathematicalModeling/2026mcm/data/processed/dwts_season_features.csv'))
❌ Unexpected error: name 'pd' is not defined


Traceback (most recent call last):
  File "C:\Windows\Temp\ipykernel_17980\3002375752.py", line 9, in <module>
    weekly = pd.read_csv(q0_out.weekly_panel_csv)
             ^^
NameError: name 'pd' is not defined


## 1) 运行 Q0

产物：写入 `data/processed/`。

In [ ]:
from mcm2026.pipelines import mcm2026c_q0_build_weekly_panel

q0_out = mcm2026c_q0_build_weekly_panel.run()
q0_out


## 2) 查看输出表头

论文手重点：这两张表是后续所有建模与出图的“统一数据口径”。

In [ ]:
weekly = pd.read_csv(q0_out.weekly_panel_csv)
features = pd.read_csv(q0_out.season_features_csv)

weekly.head(3), features.head(3)


## 3) 快速“肉眼检查”：数据有没有长得像我们想要的那样？

下面这个检查非常适合论文手快速理解两张表：
- **行数/列数（shape）**：大概数据规模是什么量级
- **season/week 覆盖范围**：是不是覆盖了我们需要的季度与周数
- **active contestants 随周变化曲线**：直观上应该是“越到后面人越少”（基本单调下降）

如果这里就出现异常（比如某周 active 人数突然变多），后面所有题都可能被污染。

## 常见问答（FAQ）

### Q0 是不是“只是改格式”？
不是。

它的核心价值是 **统一口径**：
- 后面每一题都要用到“每周谁在场、评委怎么打分、谁离开了”
- 如果每个脚本自己算一套口径，很容易出现矛盾（论文会崩）

### 为什么一定要做 weekly panel（长表）？
因为建模/画图基本都是按 “(season, week, contestant)” 这个粒度做循环与统计。

wide 表对人更友好，但对程序来说：
- 不好 groupby
- 不好 merge
- 不好做 sanity check

### 我怎么判断 Q0 做对了？
最简单的 3 个检查：
- `weekly_panel` 行数是“周×选手”的量级（比 season_features 大很多）
- `judge_score_pct` 在同一 (season, week) 下，活跃选手的 pct 加起来应接近 1
- 每个季度 `n_active` 随 week 基本下降（越到后面人越少）

### 如果 Q0 有问题，会影响后面哪一题？
**所有题**。

因为 Q1/Q2/Q3/Q4 的输入都来自 processed 数据。只要 Q0 在某些 week/season 口径错了，后面结论就会被“系统性污染”。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

print('weekly_panel shape:', weekly.shape)
print('season_features shape:', features.shape)

print('season range:', int(pd.to_numeric(weekly['season'], errors='coerce').min()), '→', int(pd.to_numeric(weekly['season'], errors='coerce').max()))
print('week range:', int(pd.to_numeric(weekly['week'], errors='coerce').min()), '→', int(pd.to_numeric(weekly['week'], errors='coerce').max()))

# 1) 看一眼我们最关心的几列长什么样
cols_preview = [
    'season',
    'week',
    'celebrity_name',
    'judge_score_total',
    'judge_score_pct',
    'judge_rank',
    'active_flag',
    'exit_type',
]
cols_preview = [c for c in cols_preview if c in weekly.columns]
weekly[cols_preview].head(8)

# 2) “每周还剩多少人在场”应该整体随周数下降（每个季度单调下降或大致下降）
w = weekly.copy()
if 'active_flag' in w.columns:
    w['active_flag'] = w['active_flag'].astype(bool)

n_active = (
    w[w['active_flag']]
    .groupby(['season', 'week'], sort=True)['celebrity_name']
    .nunique()
    .reset_index(name='n_active')
)

seasons = sorted(pd.to_numeric(n_active['season'], errors='coerce').dropna().astype(int).unique())
if seasons:
    sample = [seasons[0], seasons[len(seasons)//2], seasons[-1]]
else:
    sample = []

fig, ax = plt.subplots(figsize=(10, 4))
for s in sample:
    g = n_active[n_active['season'].astype(int) == int(s)].sort_values('week')
    ax.plot(g['week'], g['n_active'], marker='o', linewidth=2, label=f'Season {s}')

ax.set_title('Quick sanity: active contestants should decrease over weeks')
ax.set_xlabel('Week')
ax.set_ylabel('# Active contestants')
ax.grid(True, alpha=0.3)
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
weekly.columns.tolist()


In [ ]:
features.columns.tolist()
